# General information
This code is the central structure of `The Conference Calculator`. 

The sections of the basic version of the tool are:

1. Basic inputs (starting point, location of the conference).
2. Query conference database.
3. Route calculation for different modes of transportation (car, train, plane).
4. Carbon footprint calculation for different modes of transportation (car, train, plane).
5. Importance of the conference.
6. Outputs (2D plot).

## 0. Importations

In [1]:
import inquirer as iq
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import urllib.request
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
%matplotlib qt

## 1. Basic inputs
Questions asked (for now!):
1. What's your starting point?
2. Where is the conference?

In [2]:
list_of_cities = {
    'London':'St Pancras International',
    'Paris': 'Gare du Nord',
    'Brussels': 'Brussel-Centraal',
    'Amsterdam': 'Amsterdam Centraal',
    'Luxembourg': 'Luxembourg Railway Station',
    'Bern': 'Bern Railway Station',
    'Berlin':'Berlin Hauptbahnhof',
    'Copenhagen':'Copenhagen Central Station',
    'Prague': 'Prague Main Railway Station',
    'Ljubljana':'Ljubljana Train Station',
    'Vienna': 'Vienna International Busterminal (VIB)',
    'Oslo':'Oslo Central Station',
    'Bratislava': 'Bratislava Hlavna',
    'Madrid': 'Madrid Atocha',
    'Zagreb':'Zagreb Glavni Kolod Train Station',
    'Rome': 'Roma Termini',
    'Budapest': 'Budapest Keleti',
    'Warsaw': 'Warszawa Centralna railway station',
    'Stockholm':'Stockholm Central Station',
    'Sarajevo':'Sarajevo Main Railway station',
    'Lisbon':'Rossio Railway Station', 
    }

print("LIST OF AVAILABLE CITIES")
print("--------------------------------------")
for city in list_of_cities.keys(): print(city)
print("--------------------------------------")

# Departure city (starting point)
starting_point = input("Please enter your departure city: ")
if starting_point not in list_of_cities.keys(): 
    raise Exception("City not available! Enter another city.")

# Destination city (destination point)
destination_point = input("Please enter your destination city: ")
if destination_point not in list_of_cities.keys(): 
    raise Exception("City not available! Enter another city.")

# Create dictionary with both cities
journey_info = {'starting_point': starting_point, 'destination_point': destination_point}
journey_info

LIST OF AVAILABLE CITIES
--------------------------------------
London
Paris
Brussels
Amsterdam
Luxembourg
Bern
Berlin
Copenhagen
Prague
Ljubljana
Vienna
Oslo
Bratislava
Madrid
Zagreb
Rome
Budapest
Warsaw
Stockholm
Sarajevo
Lisbon
--------------------------------------


{'starting_point': 'London', 'destination_point': 'Paris'}

## 2. Query conference database
TODO: Add Lorenzo code

## 3. Route calculation
TODO: Add Iona code

Output: add distance from Google Maps API to `journey_info` for car, train and plane

In [3]:
############# ADD CODE

journey_info['distance_plane'] = 500.
journey_info['distance_car'] = 300.
journey_info['distance_train'] = 450.

## 4. Carbon footprint calculation
TODO: Add Tsvet & Jeremie code

Output: add carbon footprint from Google Maps API to `journey_info` for car, train and plane

In [4]:
############# ADD CODE

journey_info['carbon_footprint_plane'] = 500. * 10
journey_info['carbon_footprint_car'] = 300. * 5
journey_info['carbon_footprint_train'] = 450. * 2

## 5. Importance of the conference
- *Are you presenting?* $\rightarrow$ talk, poster, invited talk, workshop lecture, etc.
- *How often is the conference held?* $\rightarrow$ e.g., every four years, unique conference, annually.
- *How specific is the conference to your research project?* $\rightarrow$ broad, specific, expert level?
- *Is there a relevant workshop/session you want/need to attend?*
- *Is the conference offered in hybrid mode?*
- *Are you an Early Career Researcher?*

In [31]:
class conferenceImportance:
    def __init__(self):
        self.conference_importance = {'dictionaries': {},
                                      'information': {}}
        
        dict_ECR = {"Yes": 1.5,
                    "No": 1.}
        
        dict_presenting = { "I'm not presenting anything": 1,
                            'I was SELECTED to present a POSTER': 5,
                            'I was INVITED to present a POSTER': 6,
                            'I was SELECTED to give a TALK (less than 15 minutes including questions)': 7,
                            'I was INVITED to give a TALK (less than 15 minutes including questions)': 8,
                            'I was SELECTED to give a TALK (15 minutes or more including questions)': 8,
                            'I was INVITED to give a TALK (15 minutes or more including questions)': 9,
                            'I was INVITED to chair a session': 5,
                            'I was INVITED to be a panelist': 8,
                            'I was INVITED to give a workshop': 10,
                            'Other': None}
    
        dict_expertise = { 'Broad (e.g., all STEM conference, outreach conference)': 4,
                            'Field-specific (e.g., physics conference)': 5,
                            'Subfield-specific (e.g., astrophysics conference)': 6,
                            'Topic-specific (e.g., black hole conference)': 8,
                            'Research-specific (e.g., black hole feedback conference)': 10,
                            'Other': None}

        self.conference_importance['dictionaries']['dict_ECR'] = dict_ECR
        self.conference_importance['dictionaries']['dict_presenting'] = dict_presenting
        self.conference_importance['dictionaries']['dict_expertise'] = dict_expertise

    def asking_question_from_dict(self, dict_name):

        for idx, key in enumerate(self.conference_importance['dictionaries'][dict_name].keys()):
            print(idx+1, '-', key)

        user_info = int(input("Please select an option (1-{}):".format(len(self.conference_importance['dictionaries'][dict_name].keys()))))    
        user_info_score = self.conference_importance['dictionaries'][dict_name][list(self.conference_importance['dictionaries'][dict_name].keys())[user_info-1]]   

        return user_info_score

    def ask_ECR(self):
        print('\nQUESTION: Are you an Early Career Researcher (e.g., undergraduate student, postgraduate student, postdoctoral researcher, research assistant)?')
        user_info_ECR_score = self.asking_question_from_dict('dict_ECR')
        self.conference_importance['information']['info_ECR'] = user_info_ECR_score

    def ask_presenting(self):
        print('\nQUESTION: Are you presenting at the conference?')
        user_info_presenting_score = self.asking_question_from_dict('dict_presenting')

        if user_info_presenting_score == None:
            user_info_presenting_score = int(input('Please evaluate your contribution on a scale from 1 to 10: '))

        self.conference_importance['information']['info_presenting'] = user_info_presenting_score * self.conference_importance['information']['info_ECR']
    
    def ask_expertise(self):
        print('\nQUESTION: How specific is the conference to your field of expertise?')
        user_info_expertise_score = self.asking_question_from_dict('dict_expertise')

        if user_info_expertise_score == None:
            user_info_expertise_score = int(input('Please evaluate the importance (in terms of expertise) from 1 to 10: '))

        self.conference_importance['information']['info_expertise'] = user_info_expertise_score

    def main(self):
        self.ask_ECR()
        self.ask_presenting()    
        self.ask_expertise()

        return self.conference_importance

conference_importance = conferenceImportance().main()
        


QUESTION: Are you an Early Career Researcher (e.g., undergraduate student, postgraduate student, postdoctoral researcher, research assistant)?
1 - Yes
2 - No

QUESTION: Are you presenting at the conference?
1 - I'm not presenting anything
2 - I was SELECTED to present a POSTER
3 - I was INVITED to present a POSTER
4 - I was SELECTED to give a TALK (less than 15 minutes including questions)
5 - I was INVITED to give a TALK (less than 15 minutes including questions)
6 - I was SELECTED to give a TALK (15 minutes or more including questions)
7 - I was INVITED to give a TALK (15 minutes or more including questions)
8 - I was INVITED to chair a session
9 - I was INVITED to be a panelist
10 - I was INVITED to give a workshop
11 - Other

QUESTION: How specific is the conference to your field of expertise?
1 - Broad (e.g., all STEM conference, outreach conference)
2 - Field-specific (e.g., physics conference)
3 - Subfield-specific (e.g., astrophysics conference)
4 - Topic-specific (e.g., black

In [5]:
# Question: Are you an Early Career Researcher?


# Question: Are you presenting?
presentation_type = {"I'm not presenting": 0., 
                     "I'm presenting a poster": 5.,
                     "I'm presenting a talk": 8.,
                     "I'm giving a workshop": 15.,
                     "I was invited to give a talk": 12.}

presentation_type = input("Please enter your departure city: ")
if starting_point not in list_of_cities.keys(): 
    raise Exception("City not available! Enter another city.")

journey_info['importance'] = 6.

## 6. Outputs
### 6.1 UK average carbon footprint (used for plotting)

### 6.1 2D plot

In [41]:
fig, ax = plt.subplots()

def getImage(path):
   return OffsetImage(plt.imread(path, format="png"), zoom=0.15)

def mean(a, b):
    return (a+b)/2

paths = ['images\\plane.png', 'images\\car.png', 'images\\train.png']

#with plt.xkcd():
average_UK = 12700 # tonnes CO2e
average_UK = 4000 # tonnes CO2e

percentages = np.array([0., 0.5, 1., 1.5, 2., 5.])
cf_percentages = average_UK * percentages

list_c = ['red', 'green', 'blue', 'orange', 'purple']

for idx in range(len(cf_percentages)-1):
    print(cf_percentages[idx], cf_percentages[idx+1])
    ax.axvspan(cf_percentages[idx], cf_percentages[idx+1], alpha=0.2, color=list_c[idx])
    ax.text(mean(cf_percentages[idx], cf_percentages[idx+1]), 8, str(percentages[idx])+'–'+str(percentages[idx+1])+"x", color=list_c[idx], ha='center', fontsize=15)

for idx, transport in enumerate(['plane', 'car', 'train']):
   print(idx, transport)
   print(journey_info['carbon_footprint_{}'.format(transport)], journey_info['importance'])
   ab = AnnotationBbox(getImage(paths[idx]), (journey_info['carbon_footprint_{}'.format(transport)], journey_info['importance']), frameon=False)
   ax.add_artist(ab)

plt.xlim(0, 6000)
plt.ylim(0, 10)
plt.xlabel("Carbon footprint (tonnes CO2e)", fontsize=15)
plt.ylabel("Importance of the conference", fontsize=15)
plt.title('Conference in {} (departure from {})'.format(journey_info['destination_point'], journey_info['starting_point']), fontsize=15)
plt.show()

0.0 2000.0
2000.0 4000.0
4000.0 6000.0
6000.0 8000.0
8000.0 20000.0
0 plane
5000.0 6.0
1 car
1500.0 6.0
2 train
900.0 6.0
